# Wallach Clarification

What did Hannah Wallach say?

 * Well she's thinking of T topics, and I guess ? words so here component distribution is  $\Phi \in \mathbb{R}^{T \times ?}$ with prior $\text{Dir}(\phi_t; \beta \boldsymbol{n})$
 * For each of the $D$ documents there's a topic distribution $\theta_d$ with prior $\text{Dir}(\theta_d; \alpha \boldsymbol{m})$

Finally, she notes the Polya identity, allowing the marginalisation of most parameters.

She then moves out into how to evaluate the probability of some held out documents $W$ given some training documents $W'$ which is

$$
p(W | W') = \int d\Phi d\alpha d\boldsymbol{m}
             \text{ } p(W | \Phi, \alpha, \boldsymbol{m}) \text{ } p(\Phi, \alpha, \boldsymbol{m}|W')
$$

The thing to note here is she has already margnalised out $\Theta$ for the new documents. She assumes you learn the "global" parameters -- priors and component distribution -- and then fix these and use them to evaluate the new documents

> So we have to think about what we're doing here. A mixture model is a good case. You can just directly evaluate the log likelihood $p(w|\alpha, \Phi) = \sum_k p(w | \phi_k)p(z=k|\alpha)$. Or you can determine the posterior over clusters and use that to evaluate... except that it doesn't decompose $p(w|\alpha, \Phi) = \sum_k p(w, z=k|\alpha, \Phi) = p(z=k|w, \alpha, \Phi)p(w|\ldots)$. But it seems obvious to see how well you can "explain" documents: this is what doc-completion does. Hence it should be introduced in the clustering section. It's also a good metric to use if you want to consider the predictive ability to, e.g. predict hashtags.

Now either way, you have to make a choice about your parameters. Are you using the _distribution_ over the parameters, or are you just taking a point estimate?

1. Drawing samples from the parameter posterior and taking an average to evaluate the integral, i.e.  $\mathbb{E}_{p(\Phi, \alpha, \boldsymbol{m}|W')}\left[ p(W | \Phi, \alpha, \boldsymbol{m}) \right]$. 
    * Stick a log in that expectation and you can start thinking about a variational approximation.
2. Taking a point estimate of -- I guess $\Phi, \alpha, \boldsymbol{m}$ -- and then use that to approximate

The paper is concerned with point estimates. So where's the uncertainty.... Apparently its in $p(\boldsymbol{w}_d | \alpha \boldsymbol{m}, \Phi)$

The next thing is that we've marginalised out $\theta$ for each of the inference documents. We need this too. If you hold $\Phi$ fixed (and so let it be found by any inference method), you can use Gibbs sampling to quickly get a distribution over $z$ and thereby, $\theta$.

 * This is used by many methods she describes, being: FIXME
 * There are other methods that do not require this, being: FIXME
 


### Estimating $p(w|\Phi, a \boldsymbol{m})$



#### Using Importance Sampling


Hence there are two options:

Directly sample $\theta \sim Dir(\alpha \boldsymbol{m})$ and average over all settings. But importance sampling doesn't work well in high-dimensions: it has high-variance, indeed, infinite variance with real-values high-dim values.

The other is to choose a proposal distribution and weight such samples in the usual importance-sampling way. The proposal distribution is in fact a method for evaluating the posterior $p(z|w, \alpha \boldsymbol{m}, \Phi)$

$$
\theta^0 \propto \left(\alpha \boldsymbol{m}\right) \text{.* } \Phi_{\cdot, w_{n}} 
$$

Which is just the prior over topics and the probability of words under each topic, i.e. $p(z = k| w, \Phi, \alpha \boldsymbol{m}) \propto p(w|Phi, z=k)p(z=k| \alpha \boldsymbol{m})$

To draw samples, simply iterate
$$
\begin{align*}
\text{for }& s = 0 \ldots S \\
 & z_n^{(s)} \sim \text{Mul}(\theta^{(s)}, 1) \\
 & \theta^{(s+1)} \propto \left(\alpha \boldsymbol{m} + \sum_{n' \neq n} \theta^{(s)} \text{.* } \boldsymbol{\bar{z}}_{n'}\right) \Phi_{\cdot, w_{n}}
\end{align*}
$$

(Recall that in more normal notation $\alpha \boldsymbol{m} = \boldsymbol{\alpha}$ and parameterises the prior. Also $z_n$ is the scalar and $\bar{\boldsymbol{z}}_n$ is the indicator vector.

#### Use the Harmonic Mean

Use Gibbs sampling to get a _posterior_ distribution over $z_n^s$.

Then instead of using that to materlise an estimate of $\theta$ (WHY), use it directly to figure out $p(w | \alpha \boldsymbol{m}, \Phi)$